In [ ]:
import ibis
from ibis import _
import pathlib
import os
from osgeo import gdal
from cng.utils import *
from cng.h3 import *

gdal.DontUseExceptions()
os.environ['AWS_VIRTUAL_HOSTING'] = 'FALSE'

# from cng.zonal import * 
con = ibis.duckdb.connect("duck.db",extensions = ["spatial", "h3"])
install_h3()
set_secrets(con)

input_url = "/vsicurl/https://minio.carlboettiger.info/public-carbon/cogs/vulnerable_c_total_2018.tif"

#con.raw_sql("SET threads=1;") # Reducing threads reduces RAM use of duckdb
#con.raw_sql("SET memory_limit = '58GB';") # Reducing threads reduces RAM use of duckdb

In [ ]:
df = con.read_parquet("s3://public-grids/hex/h0.parquet").mutate(h0 = _.h0.lower()).execute()


In [ ]:
#set_secrets(con, "", "", "s3.amazonaws.com", "overturemaps-us-west-2", 'vhost')
#overture = con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', filename=True, hive_partitioning=1)
#wkt = overture.filter(_.subtype=="region").filter(_.region == "US-CA").select(_.geometry).execute().geometry[0].wkt

In [ ]:
%%time
# CA in 17 seconds
input_url = "/vsicurl/https://minio.carlboettiger.info/public-carbon/cogs/vulnerable_c_total_2018.tif"
dest = "test_carbon.xyz"
gdal.Warp(dest, input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)

In [ ]:
%%time
i = 100
wkt = df.geom[i]
h0 = df.h0[i]
zoom = 8
gdal.Warp("tmp-carbon.xyz", input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)


In [ ]:
%%time
(con
  .read_csv("tmp-carbon.xyz", 
            delim = ' ', 
            columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'})
  .mutate(h0 = h3_latlng_to_cell_string(_.Y, _.X, zoom),
          h8 = h3_latlng_to_cell_string(_.Y, _.X, zoom))
  .mutate(Z = ibis.ifelse(_.Z == 65535, None, _.Z)) 
 .to_parquet("test.parquet")
  #.to_parquet(f"s3://public-carbon/hex/vulnerable-carbon/h0={h0}/vulnerable-total-carbon-2018-h{zoom}.parquet")
)
pathlib.Path("tmp-carbon.xyz").unlink()

con.read_parquet("test.parquet")

In [ ]:

#see `gdal.WarpOptions?` for details. Also see resampling options in warper for large data
# NOTE! dest given before input!
for i in range(df.shape[0]):
    wkt = df.geom[i]
    h0 = df.h0[i]
    zoom = 8
    print(h0)

    try:
        gdal.Warp("tmp-carbon.xyz", input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)
    
        (con
          .read_csv("tmp-carbon.xyz", 
                    delim = ' ', 
                    columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'})
          .mutate(h0 = h3_latlng_to_cell_string(_.Y, _.X, zoom),
                  h8 = h3_latlng_to_cell_string(_.Y, _.X, zoom))
          .mutate(Z = ibis.ifelse(_.Z == 65535, None, _.Z)) 
          .to_parquet(f"s3://public-carbon/hex/vulnerable-carbon/h0={h0}/vulnerable-total-carbon-2018-h{zoom}.parquet")
        )
        pathlib.Path("tmp-carbon.xyz").unlink()
    except Exception as e:
        print(f"Error processing item {i}: {e}")
    



In [ ]:
con.read_parquet("s3://public-carbon/hex/vulnerable-carbon/")